# Downloading and coverting ERA5 files

Information on the fifth generation ECMWF atmospheric reanalysis of the global climate ERA5 is found at:
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

# 1) Requesting ERA5 data

a) Go to the ERA5 hourly data on single levels from 1979 to present (https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form)

b) In product type select Reanalysis

c) In Variable select Popular -> 10m u-component of wind, 10m v-component of wind, and Mean sea level pressure.

d) In this example we will prepare the data for Hurricane Arthur, from 06/27/2014 to 07/09/2014. Therefore, we will have to download 2 ERA5 files, one from 06/27/2014 to 06/30/2014 and another from 07/01/2014 to 07/09/2014. 

First, we will download the data from 06/27/2014 to 06/30/2014. To do that we just need to select the Year, Month and Day accordingly. We will work with inputs every 6 hours, so select the Time: 00:00, 06:00, 12:00, and 18:00.

e) In the Geographical area tab select “Whole available region”

f) Select the GRIB format in the Format tab.

g) Click on “Show API request” and save the code.


# 2) Downloading ERA5 data

Importing the necessary lybraries:


cfgrib is installed as:
conda install -c conda-forge cfgrib
xarray is installed as:
conda install -c conda-forge xarray dask netCDF4 bottleneck

In [1]:
import sys,getopt
import pandas as pd
import numpy as np
import pathlib as pl
import numpy.core.multiarray 
import netCDF4 as nc4
import xarray as xr
import scipy.interpolate
import cdsapi

Paste the API request and start downloading:

In [2]:
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure',
        ],
        'year': '2014',
        'month': '06',
        'day': [
            '27', '28', '29',
            '30',
        ],
        'time': [
            '00:00', '06:00', '12:00',
            '18:00',
        ],
    },
    'download.grib')

2020-04-09 14:14:09,064 INFO Welcome to the CDS
2020-04-09 14:14:09,068 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-04-09 14:14:10,938 INFO Request is completed
2020-04-09 14:14:10,941 INFO Downloading http://136.156.132.198/cache-compute-0003/cache/data3/adaptor.mars.internal-1586349034.2352884-14786-33-c4c25e88-3359-4a03-93f2-1341c4d937ff.grib to download.grib (95.1M)
2020-04-09 14:14:20,799 INFO Download rate 9.6M/s                                                                      


Result(content_length=99676800,content_type=application/x-grib,location=http://136.156.132.198/cache-compute-0003/cache/data3/adaptor.mars.internal-1586349034.2352884-14786-33-c4c25e88-3359-4a03-93f2-1341c4d937ff.grib)

# 3) Converting the .grib file into .22 (NWS=6)

Set the path to where the .grib data was saved:

Open the .grib file:

In [3]:
path = pl.Path(r'C:\Users\fcassalh')
file = 'download.grib'
grib = xr.open_dataset(path / file, engine='cfgrib')

Extract the variables of interest:

In [5]:
u = grib.data_vars['u10'].values[:]
v = grib.data_vars['v10'].values[:]
prmsl = grib.data_vars['msl'].values[:]
lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values
time = grib.coords['time'].values[:]

Create boundaries that cover the extent of your mesh (in our case the North Atlantic and the Caribean):

In [7]:
lat1,lat2 = 7,60
lon1,lon2 = -100, -58

Create a grid in using the format expected for a NWS = 6 in 3 columns (uwind, vwind, pressure):

In [8]:
temp,y,x = u.shape
u2,v2,p2 = [],[],[]
datay,datax = [], []

Start populating the grid with data from the .grib file:

In [9]:
for t in time:
    for i in range(0,y):
        for ii in range(0,x):
            if (lat2>=lat[i]>lat1) and ((lon2+360)>=lon[ii]>(lon1+360)):
                u2.append(np.round(u[t,i,ii],3))
                v2.append(np.round(v[t,i,ii],3))
                p2.append(prmsl[t,i,ii])
                datay.append(i)
                datax.append(ii)

Save the grid in the correct format:

In [10]:
df = pd.DataFrame({'u':np.round(u2,2),'v':np.round(v2,2),'p':np.round(p2,2)})
df.to_csv('Arthur_part1.22', sep='\t',index=False,header = False)  

Repeat steps 1, 2, and 3 for the second part of the storm

Make sure that when you repeat the process for the other part of the storm (07/01/2014 to 07/09/2014) you change the name of the output file.

Use a text editor (Sublime for instance) to copy the 'arthur_2.22' file at the end of the 'arthur_1.22' file. This complete file refers to the whole storm.

Check the extent of your grid:

In [11]:
temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2+360>=lon) & (lon>lon1+360)]
len(temp),len(temp2)

(212, 168)

# Updating the fort.15 file:

Change the line YYYY MM DD HH24 to:

212 168 60.0 -100.0 0.25 0.25 21600 3600 ! YYYY MM DD HH24 StormNumber BLAdj 

where, 212 is the number of values in Y, 168 is the number of values in X, 60 is the maximum Y, -100 is the minimum X, 0.25 are the X and Y increments, and 21600 is the time interval in seconds (6hours).